# Part 1: Scraping the data from wikipedia

### Import required libraries

In [1]:
import pandas as pd

### Scrape Wikipedia for the data using pandas read_html function

Define url variable using correct wikipedia address, then read into pandas dataframe and show top 10 rows to examine

In [2]:
url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"

df=pd.read_html(url)[0]

df.head(10)

Postal Code           Borough                                Neighbourhood
0         M1A      Not assigned                                 Not assigned
1         M2A      Not assigned                                 Not assigned
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
7         M8A      Not assigned                                 Not assigned
8         M9A         Etobicoke      Islington Avenue, Humber Valley Village
9         M1B       Scarborough                               Malvern, Rouge

### Process and clean data

First discard / ignore any rows that say 'Not Assigned'. One assumption I have made here is that any Neighborhood cells that contain 'Not Assigned' match the Borough cells that contain 'Not Assigned'. Manual evaluation of the dataframe confirmed this to be true, so no further code or action is required to satisy the assignment requirements.

In [3]:
df=df[df.Borough != 'Not assigned']

Sort by Postal Code and Borough to clean and tidy up the dataframe, then reset the index to represent the new order and missing rows. Finally, show the top 10 rows to examine the new cleaned up dataframe.

In [4]:
df = df.sort_values(by=['Postal Code','Borough'])

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

df.head(10)

Postal Code      Borough                                    Neighbourhood
0         M1B  Scarborough                                   Malvern, Rouge
1         M1C  Scarborough           Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough                Guildwood, Morningside, West Hill
3         M1G  Scarborough                                           Woburn
4         M1H  Scarborough                                        Cedarbrae
5         M1J  Scarborough                              Scarborough Village
6         M1K  Scarborough      Kennedy Park, Ionview, East Birchmount Park
7         M1L  Scarborough                  Golden Mile, Clairlea, Oakridge
8         M1M  Scarborough  Cliffside, Cliffcrest, Scarborough Village West
9         M1N  Scarborough                      Birch Cliff, Cliffside West

### Use .shape method to print the number of rows in the dataframe

In [5]:
df.shape

(103, 3)

# Part 2: Add Latitudes and Longitudes to dataframe

### Use CSV file to import manually downloaded data

Define path as location of CSV file on local machine, then read into new dataframe using the pandas read_csv function. Show the top 10 rows to examine the dataframe.

In [6]:
path='D:\Downloads\Geospatial_Coordinates.csv'

df_geospatial = pd.read_csv(path)
df_geospatial.head(10)

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
5         M1J  43.744734 -79.239476
6         M1K  43.727929 -79.262029
7         M1L  43.711112 -79.284577
8         M1M  43.716316 -79.239476
9         M1N  43.692657 -79.264848

Merge the dataframes using the pandas merge function, on the Postal. Then show top 10 rows to examine the results.

In [7]:
df_merged = pd.merge(df, df_geospatial, on='Postal Code')
df_merged.head(10)

Postal Code      Borough                                    Neighbourhood  \
0         M1B  Scarborough                                   Malvern, Rouge   
1         M1C  Scarborough           Rouge Hill, Port Union, Highland Creek   
2         M1E  Scarborough                Guildwood, Morningside, West Hill   
3         M1G  Scarborough                                           Woburn   
4         M1H  Scarborough                                        Cedarbrae   
5         M1J  Scarborough                              Scarborough Village   
6         M1K  Scarborough      Kennedy Park, Ionview, East Birchmount Park   
7         M1L  Scarborough                  Golden Mile, Clairlea, Oakridge   
8         M1M  Scarborough  Cliffside, Cliffcrest, Scarborough Village West   
9         M1N  Scarborough                      Birch Cliff, Cliffside West   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476  
5  43.744734 -79.239476  
6  43.727929 -79.262029  
7  43.711112 -79.284577  
8  43.716316 -79.239476  
9  43.692657 -79.264848

# Part 3: Explore and Cluster the neighborhoods in Toronto

### For this task I am going to work with just the boroughs that contain the word Toronto, as that is our location of interest

Import additional required libraries

In [8]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Filter our table just for boroughs that contain the word 'Toronto', then examine the top 10 rows to take a look.

In [9]:
df_toronto = df_merged[df_merged['Borough'].str.contains('Toronto')]
df_toronto.reset_index(drop=True, inplace=True)
df_toronto.head(10)

Postal Code          Borough  \
0         M4E     East Toronto   
1         M4K     East Toronto   
2         M4L     East Toronto   
3         M4M     East Toronto   
4         M4N  Central Toronto   
5         M4P  Central Toronto   
6         M4R  Central Toronto   
7         M4S  Central Toronto   
8         M4T  Central Toronto   
9         M4V  Central Toronto   

                                       Neighbourhood   Latitude  Longitude  
0                                        The Beaches  43.676357 -79.293031  
1                       The Danforth West, Riverdale  43.679557 -79.352188  
2                     India Bazaar, The Beaches West  43.668999 -79.315572  
3                                    Studio District  43.659526 -79.340923  
4                                      Lawrence Park  43.728020 -79.388790  
5                                   Davisville North  43.712751 -79.390197  
6                  North Toronto West, Lawrence Park  43.715383 -79.405678  
7                                         Davisville  43.704324 -79.388790  
8                        Moore Park, Summerhill East  43.689574 -79.383160  
9  Summerhill West, Rathnelly, South Hill, Forest...  43.686412 -79.400049

Examine the size of the dataframe using the .shape method

In [10]:
df_toronto.shape

(40, 5)

### Create a map of Toronto with neighborhoods superimposed on top

Define coordinates of Toronto (I could not install the geopy library on my machine so just obtained these manually)

In [11]:
latitude = 43.6532 
longitude = -79.3832

Create map of Toronto using above latitude and longitude values

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

Add markers to map

In [13]:
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define my Foursquare Credentials and Version

In [14]:
CLIENT_ID = 'CCKOQHSERN1KJLQR44C4RZBZ4NH4QO4ELA3EU4FBLIMKBEGL'
CLIENT_SECRET = 'E0FLMAGE44ZKOJ44VXSHPX2J4NZDCPX34NJRKQIFDKEEJ2HO' 
VERSION = '20180605'
LIMIT = 100 

### Create a function to iteratively explore the Neighborhoods in Toronto

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called toronto_venues

In [16]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
Runny

### Examine the size and print out a sample of the resulting dataframe

In [17]:
print(toronto_venues.shape)
toronto_venues.head()

(1612, 7)


Neighborhood  Neighborhood Latitude  \
0                   The Beaches              43.676357   
1                   The Beaches              43.676357   
2                   The Beaches              43.676357   
3                   The Beaches              43.676357   
4  The Danforth West, Riverdale              43.679557   

   Neighborhood Longitude                               Venue  Venue Latitude  \
0              -79.293031                   Glen Manor Ravine       43.676821   
1              -79.293031  The Big Carrot Natural Food Market       43.678879   
2              -79.293031                 Grover Pub and Grub       43.679181   
3              -79.293031                       Upper Beaches       43.680563   
4              -79.352188                       MenEssentials       43.677820   

   Venue Longitude     Venue Category  
0       -79.293942              Trail  
1       -79.297734  Health Food Store  
2       -79.297215                Pub  
3       -79.292869       Neighborhood  
4       -79.351265     Cosmetics Shop

Check how many venues were returned for each neighborhood

In [18]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            60   
Brockton, Parkdale Village, Exhibition Place                           22   
Business reply mail Processing Centre, South Ce...                     16   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     61   
Christie                                                               16   
Church and Wellesley                                                   77   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             37   
Davisville North                                                        7   
Dufferin, Dovercourt Village                                           16   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          25   
India Bazaar, The Beaches West                                         18   
Kensington Market, Chinatown, Grange Park                              63   
Lawrence Park                                                           4   
Little Portugal, Trinity                                               42   
Moore Park, Summerhill East                                             4   
North Toronto West, Lawrence Park                                      20   
Parkdale, Roncesvalles                                                 14   
Queen's Park, Ontario Provincial Government                            30   
Regent Park, Harbourfront                                              44   
Richmond, Adelaide, King                                               94   
Rosedale                                                                4   
Roselawn                                                                3   
Runnymede, Swansea                                                     37   
Runnymede, The Junction, Weston-Pellam Park, Ca...                      5   
St. James Town                                                         80   
St. James Town, Cabbagetown                                            45   
Stn A PO Boxes                                                         99   
Studio District                                                        36   
Summerhill West, Rathnelly, South Hill, Forest ...                     14   
The Annex, North Midtown, Yorkville                                    20   
The Beaches                                                             4   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         32   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             60   
Brockton, Parkdale Village, Exhibition Place                            22   
Business reply mail Processing Centre, South Ce...                      16   
CN Tower, King and Spadina, Railway Lands, Harb...                      16   
Central Bay Street                                                      61   
Christie                                                                16   
Church and Wellesley                                                    77   
Commerce Court, Victoria Hotel                                         100   
Davisville                           

Check how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


### Analyze Each Neighborhood

Perform one hot encoding on toronto venues

In [20]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(10)

Yoga Studio  Adult Boutique  Airport  Airport Food Court  Airport Gate  \
0            0               0        0                   0             0   
1            0               0        0                   0             0   
2            0               0        0                   0             0   
3            0               0        0                   0             0   
4            0               0        0                   0             0   
5            0               0        0                   0             0   
6            0               0        0                   0             0   
7            0               0        0                   0             0   
8            0               0        0                   0             0   
9            0               0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   
5               0                0                 0                    0   
6               0                0                 0                    0   
7               0                0                 0                    0   
8               0                0                 0                    0   
9               0                0                 0                    0   

   Antique Shop  ...  Thrift / Vintage Store  Tibetan Restaurant  \
0             0  ...                       0                   0   
1             0  ...                       0                   0   
2             0  ...                       0                   0   
3             0  ...                       0                   0   
4             0  ...                       0                   0   
5             0  ...                       0                   0   
6             0  ...                       0                   0   
7             0  ...                       0                   0   
8             0  ...                       0                   0   
9             0  ...                       0                   0   

   Toy / Game Store  Trail  Train Station  Vegetarian / Vegan Restaurant  \
0                 0      1              0                              0   
1                 0      0              0                              0   
2                 0      0              0                              0   
3                 0      0              0                              0   
4                 0      0              0                              0   
5                 0      0              0                              0   
6                 0      0              0                              0   
7                 0      0              0                              0   
8                 0      0              0                              0   
9                 0      0              0                              0   

   Video Game Store  Vietnamese Restaurant  Wine Bar  Wine Shop  
0                 0                      0         0          0  
1                 0                      0         0          0  
2                 0                      0         0          0  
3                 0                      0         0          0  
4                 0                      0         0          0  
5                 0                      0         0          0  
6                 0                      0         0          0  
7                 0                      0         0          0  
8                 0                      0         0          0  
9                 0                      0         0         

Examine the new dataframe size:

In [21]:
toronto_onehot.shape

(1612, 238)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                                         Berczy Park     0.000000   
1        Brockton, Parkdale Village, Exhibition Place     0.000000   
2   Business reply mail Processing Centre, South C...     0.000000   
3   CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
4                                  Central Bay Street     0.016393   
5                                            Christie     0.000000   
6                                Church and Wellesley     0.025974   
7                      Commerce Court, Victoria Hotel     0.000000   
8                                          Davisville     0.000000   
9                                    Davisville North     0.000000   
10                       Dufferin, Dovercourt Village     0.000000   
11             First Canadian Place, Underground city     0.000000   
12    Forest Hill North & West, Forest Hill Road Park     0.000000   
13                           Garden District, Ryerson     0.000000   
14  Harbourfront East, Union Station, Toronto Islands     0.000000   
15                      High Park, The Junction South     0.000000   
16                     India Bazaar, The Beaches West     0.000000   
17          Kensington Market, Chinatown, Grange Park     0.000000   
18                                      Lawrence Park     0.000000   
19                           Little Portugal, Trinity     0.023810   
20                        Moore Park, Summerhill East     0.000000   
21                  North Toronto West, Lawrence Park     0.050000   
22                             Parkdale, Roncesvalles     0.000000   
23        Queen's Park, Ontario Provincial Government     0.033333   
24                          Regent Park, Harbourfront     0.022727   
25                           Richmond, Adelaide, King     0.000000   
26                                           Rosedale     0.000000   
27                                           Roselawn     0.000000   
28                                 Runnymede, Swansea     0.027027   
29  Runnymede, The Junction, Weston-Pellam Park, C...     0.000000   
30                                     St. James Town     0.000000   
31                        St. James Town, Cabbagetown     0.000000   
32                                     Stn A PO Boxes     0.010101   
33                                    Studio District     0.027778   
34  Summerhill West, Rathnelly, South Hill, Forest...     0.000000   
35                The Annex, North Midtown, Yorkville     0.000000   
36                                        The Beaches     0.000000   
37                       The Danforth West, Riverdale     0.023256   
38           Toronto Dominion Centre, Design Exchange     0.000000   
39                     University of Toronto, Harbord     0.031250   

    Adult Boutique  Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0         0.000000   0.0000              0.0000        0.0000           0.000   
1         0.000000   0.0000              0.0000        0.0000           0.000   
2         0.000000   0.0000              0.0000        0.0000           0.000   
3         0.000000   0.0625              0.0625        0.0625           0.125   
4         0.000000   0.0000              0.0000        0.0000           0.000   
5         0.000000   0.0000              0.0000        0.0000           0.000   
6         0.012987   0.0000              0.0000        0.0000           0.000   
7         0.000000   0.0000              0.0000        0.0000           0.000   
8         0.000000   0.0000              0.0000        0.0000           0.000   
9         0.000000   0.0000              0.0000        0.0000           0.000   
10        0.000000   0.0000              0.0000        0.0000           0.000   
11        0.000000   0.0000              0.0000        0.0000           0.000   
12        0.000000   0.0000              0.0000        0.0000           0.000   
13        0.000000   0.0000         

Examine the new size

In [23]:
toronto_grouped.shape

(40, 238)

Print each neighborhood along with the top 5 most common venues

In [24]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1        Bakery  0.05
2  Cocktail Bar  0.05
3    Restaurant  0.03
4      Beer Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4             Bar  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1         Pizza Place  0.06
2      Farmers Market  0.06
3       Garden Center  0.06
4             Brewery  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3          Boutique  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1      Sandwich Place  0.07


#### Convert into a pandas dataframe for clustering

First write a function to sort the venues in descending order.

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...    Light Rail Station   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                Bakery          Cocktail Bar            Restaurant   
1        Breakfast Spot           Coffee Shop             Pet Store   
2                Garden               Brewery                   Spa   
3        Airport Lounge      Airport Terminal         Boat or Ferry   
4        Sandwich Place    Italian Restaurant                  Café   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0           Cheese Shop              Pharmacy              Beer Bar   
1          Intersection               Stadium                Bakery   
2        Farmers Market  Fast Food Restaurant         Burrito Place   
3              Boutique               Airport    Airport Food Court   
4           Salad Place       Bubble Tea Shop          Burger Joint   

  8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  
0        Farmers Market    Seafood Restaurant              Shopping Mall  
1    Italian Restaurant            Restaurant               Climbing Gym  
2            Restaurant               Butcher              Auto Workshop  
3          Airport Gate       Harbor / Marina           Sculpture Garden  
4   Japanese Restaurant       Thai Restaurant  Middle Eastern Restaurant

# Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 2, 2, 0, 2, 2, 2, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with df_toronto to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

Postal Code          Borough                   Neighbourhood   Latitude  \
0         M4E     East Toronto                     The Beaches  43.676357   
1         M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2         M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3         M4M     East Toronto                 Studio District  43.659526   
4         M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               2                   Pub     Health Food Store   
1 -79.352188               2      Greek Restaurant           Coffee Shop   
2 -79.315572               2  Fast Food Restaurant     Food & Drink Shop   
3 -79.340923               0           Coffee Shop   American Restaurant   
4 -79.388790               3                  Park           Swim School   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                 Trail             Wine Shop      Doner Restaurant   
1    Italian Restaurant        Ice Cream Shop            Restaurant   
2          Liquor Store        Sandwich Place            Restaurant   
3                Bakery               Brewery                  Café   
4              Bus Line      Business Service             Wine Shop   

  6th Most Common Venue   7th Most Common Venue 8th Most Common Venue  \
0                 Diner          Discount Store   Distribution Center   
1             Bookstore  Furniture / Home Store    Frozen Yogurt Shop   
2    Italian Restaurant       Fish & Chips Shop            Steakhouse   
3             Gastropub    Gym / Fitness Center                 Diner   
4                 Diner    Ethiopian Restaurant           Escape Room   

  9th Most Common Venue       10th Most Common Venue  
0               Dog Run          Dumpling Restaurant  
1           Pizza Place                      Brewery  
2        Ice Cream Shop             Sushi Restaurant  
3                  Park    Middle Eastern Restaurant  
4     Electronics Store  Eastern European Restaurant

Finally, let's visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
3       East Toronto               0           Coffee Shop   
24   Central Toronto               0                  Café   
25  Downtown Toronto               0                  Café   
26  Downtown Toronto               0                  Café   
30  Downtown Toronto               0         Grocery Store   
31      West Toronto               0                Bakery   
32      West Toronto               0                   Bar   
33      West Toronto               0                  Café   
34      Toronto/York               0           Pizza Place   
35      West Toronto               0       Thai Restaurant   

   2nd Most Common Venue  3rd Most Common Venue  \
3    American Restaurant                 Bakery   
24        Sandwich Place            Coffee Shop   
25                Bakery              Bookstore   
26           Coffee Shop  Vietnamese Restaurant   
30                  Café                   Park   
31              Pharmacy            Supermarket   
32           Men's Store  Vietnamese Restaurant   
33        Breakfast Spot            Coffee Shop   
34               Brewery      Convenience Store   
35    Mexican Restaurant                   Café   

            4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
3                         Brewery                  Café             Gastropub   
24                            Pub          Burger Joint           Flower Shop   
25                            Bar   Japanese Restaurant                  Bank   
26  Vegetarian / Vegan Restaurant    Mexican Restaurant         Grocery Store   
30                     Baby Store           Coffee Shop            Restaurant   
31      Middle Eastern Restaurant                  Bank                   Bar   
32  Vegetarian / Vegan Restaurant            Restaurant                  Café   
33                      Pet Store          Intersection               Stadium   
34                       Bus Line         Grocery Store          Airport Gate   
35                      Speakeasy    Italian Restaurant           Flea Market   

   7th Most Common Venue      8th Most Common Venue  \
3   Gym / Fitness Center                      Diner   
24             BBQ Joint               Liquor Store   
25    Italian Restaurant                   Beer Bar   
26          Burger Joint             Farmers Market   
30    Italian Restaurant         Athletics & Sports   
31                  Café      Portuguese Restaurant   
32      Asian Restaurant         Miscellaneous Shop   
33                Bakery         Italian Restaurant   
34  Fast Food Restaurant         Falafel Restaurant   
35   Fried Chicken Joint  Cajun / Creole Restaurant   

        9th Most Common Venue     10th Most Common Venue  
3                        Park  Middle Eastern Restaurant  
24  Middle Eastern Restaurant             History Museum  
25                 Beer Store    Comfort Food Restaurant  
26                       Park                        Bar  
30                Candy Store                  Nightclub  
31                       Pool                  Wine Shop  
32                Record Shop                Pizza Place  
33                 Restaurant               Climbing Gym  
34                Event Space       Ethiopian Restaurant  
35        Arts & Crafts Store     Furniture / Home Store

#### This cluster appears to contain neighborhoods that contain a lot of cafes!

### Cluster 2

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
8    Central Toronto               1                  Park   
10  Downtown Toronto               1                  Park   
23   Central Toronto               1         Jewelry Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
8             Playground          Tennis Court                 Trail   
10            Playground                 Trail             Wine Shop   
23                  Park      Sushi Restaurant                 Trail   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
8                Dog Run          Dessert Shop                 Diner   
10            Donut Shop                 Diner        Discount Store   
23            Donut Shop        Discount Store   Distribution Center   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
8         Discount Store   Distribution Center             Donut Shop  
10   Distribution Center               Dog Run       Doner Restaurant  
23               Dog Run      Doner Restaurant              Wine Shop

#### This cluster appears to contain neighborhoods that contain a lot of parks, playgrounds and trails - outdoor spaces!

### Cluster 3

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0       East Toronto               2                   Pub   
1       East Toronto               2      Greek Restaurant   
2       East Toronto               2  Fast Food Restaurant   
5    Central Toronto               2                  Park   
6    Central Toronto               2        Clothing Store   
7    Central Toronto               2           Pizza Place   
9    Central Toronto               2           Coffee Shop   
11  Downtown Toronto               2           Pizza Place   
12  Downtown Toronto               2      Sushi Restaurant   
13  Downtown Toronto               2           Coffee Shop   
14  Downtown Toronto               2        Clothing Store   
15  Downtown Toronto               2           Coffee Shop   
16  Downtown Toronto               2           Coffee Shop   
17  Downtown Toronto               2           Coffee Shop   
18  Downtown Toronto               2           Coffee Shop   
19  Downtown Toronto               2           Coffee Shop   
20  Downtown Toronto               2           Coffee Shop   
21  Downtown Toronto               2           Coffee Shop   
27  Downtown Toronto               2       Airport Service   
28  Downtown Toronto               2           Coffee Shop   
29  Downtown Toronto               2           Coffee Shop   
36      West Toronto               2        Breakfast Spot   
37      West Toronto               2           Coffee Shop   
38  Downtown Toronto               2           Coffee Shop   
39      East Toronto               2    Light Rail Station   

   2nd Most Common Venue 3rd Most Common Venue  4th Most Common Venue  \
0      Health Food Store                 Trail              Wine Shop   
1            Coffee Shop    Italian Restaurant         Ice Cream Shop   
2      Food & Drink Shop          Liquor Store         Sandwich Place   
5         Breakfast Spot                 Hotel         Sandwich Place   
6            Coffee Shop           Yoga Studio    Sporting Goods Shop   
7           Dessert Shop        Sandwich Place     Italian Restaurant   
9            Supermarket                  Bank  Vietnamese Restaurant   
11           Coffee Shop                  Park             Restaurant   
12           Coffee Shop   Japanese Restaurant             Restaurant   
13                   Pub                Bakery                   Park   
14           Coffee Shop                 Hotel     Italian Restaurant   
15                  Café          Cocktail Bar             Restaurant   
16                Bakery          Cocktail Bar             Restaurant   
17        Sandwich Place    Italian Restaurant                   Café   
18                  Café            Restaurant         Clothing Store   
19              Aquarium                 Hotel                   Café   
20                 Hotel                  Café             Restaurant   
21            Restaurant                  Café                  Hotel   
27        Airport Lounge      Airport Terminal          Boat or Ferry   
28    Seafood Restaurant          Cocktail Bar             Restaurant   
29                  Café            Restaurant                  Hotel   
36             Gift Shop      Cuban Restaurant                Dog Run   
37           Pizza Place                  Café       Sushi Restaurant   
38      Sushi Restaurant                   Gym                  Diner   
39                Garden               Brewery                    Spa   

     5th Most Common Venue 6th Most Common Venue   7th Most Common Venue  \
0         Doner Restaurant                 Diner          Discount Store   
1               Restaurant             Bookstore  Furniture / Home Store   
2               Restaurant    Italian Restaurant       Fish & Chips Shop   
5        Food & Drink Shop      Department Store             Pizza Place   
6   Furniture / Home Store  Fast Food Restaurant                   Diner   
7                     Café       Thai Restaur

#### This cluster appears to contain lots of coffee shops!

### Cluster 4

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
4  Central Toronto               3                  Park   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
4           Swim School              Bus Line      Business Service   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
4             Wine Shop                 Diner  Ethiopian Restaurant   

  8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
4           Escape Room     Electronics Store  Eastern European Restaurant

#### This cluster appears to be an outlier as there is only one neighboorhood - possibly because it does not contain many cafes and coffee shops but also contains a lot of parks, but no other outdoor spaces?

### Cluster 5

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
22  Central Toronto               4  Fast Food Restaurant   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
22          Home Service                Garden                 Diner   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
22    Falafel Restaurant           Event Space  Ethiopian Restaurant   

   8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
22           Escape Room     Electronics Store  Eastern European Restaurant

#### This cluster is also another outlier - possibly becuase (unlike all other neighborhoods) it contains alot of event spaces, and also few cafes, coffee shops or outdoor spaces like parks and trails?